# Workflow
Los workflows son tecnicas que permiten mas control del proceso disminuyendo los margenes para las alucinaciones y la variabilidad de respuestas

## Disminucion de la variabilidad
Para disminuir la variabilidad de respuestas sobre una misma historia clinica se van a ir almacenando los resumenes generados por la AI en una BBDD. 
Fujo para un paciente dado, verificar si existe un resumen en la tabla *ai_resumen_evolucion*:
1. No existe resumen
    * Obtener paciente y evoluciones de la BBDD
    * Generar resumen
    * Almacenar resumen en tabla *ai_resumen_evolucion*
    * almacenar las referencias a las evoluciones usadas para ese resumen en *ai_referencia_evolucion*
2. Existe resumen
    * Si existe mas de uno, traer el ultimo resumen
    * Obtener las evoluciones del paciente
    * Obtener las referencias a las evoluciones usadas en el ultimo resumen de la tabla *ai_referencia_evolucion*
    * Verificar si las evoluciones actuales son las mismas que las usadas para el resumen
        - Son las mismas: obtiener resumen de la tabla *ai_resumen_evolucion*  
        - Son distintas: proceder al flujo del punto (1)

## Flujo de trabajo

Dado que los requerimientos son varios y requieren examen minucioso decidimos desagregar el proceso y crear un workflow de nodos con dedicación exclusiva a cada requerimiento.

Workflow:
1. Verificar si es necesario crear un nuevo resumen o mostrar el ultimo
2. obtener datos relevantes de historia perinatal
3. obtener datos relevantes de historia familiar
4. obtener datos relevantes de metricas de crecimiento
5. Cruzar datos de historia clinica con datos de historia perinatal
6. Cruzar datos de histira clinica con los de historia familiar
7. Analizar las curvas de crecimineto y cruzar esos datos con los datos de los antecedentes de enfermedades presentes en la historia clinica, los antecedentes perinatale o los antecedentes familiares
8. Analisis de correlaciones entre los cruces de datos
9. Redaccion de informe 


---

### Creando el grafo

In [ ]:
# Instalaciones
%pip install langchain langchain-openai langchain_experimental python-dotenv matplotlib numpy pandas langgraph langchain-community langchain_tavily

In [7]:
# Importaciones
from langgraph.graph import START, END, StateGraph
from IPython.display import Image, display
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model

In [11]:
# configurar el parser de salida
from typing import Optional
from pydantic import BaseModel, Field

class Paciente(BaseModel):
    id_paciente: int = Field(...)
    apellido: str = Field(...)
    nombre: str = Field(...)
    fecha_nac: str = Field(...)
    sexo: str = Field(...)
    edad: str = Field(...)
    dni: str = Field(...)
    localidad: str = Field(...)
    obra_social: str = Field(...)
    afiliado_nro: str = Field(...)
    telefono: str = Field(...)
    telefono_numero: str = Field(...)
    email: str = Field(...)
    especialidad: str = Field(...)
    diagnostico: str = Field(...)
    enfermedad_base: str = Field(...)
    ant_perinatales: str = Field(...)
    ant_familiares: str = Field(...)
    registro: str = Field(...)
    fecha_registro: str = Field(...)
    
class Evolucion(BaseModel):
    id: int = Field(...)
    id_paciente: int = Field(...)
    fecha: str = Field(...)
    edad: int = Field(...)
    uni_edad: Optional[str] = Field(None)
    edad_anios: float = Field(...)
    edad_texto: str = Field(...)
    peso: float = Field(...)
    talla: float = Field(...)
    imc: float = Field(...)
    pc: float = Field(...)
    motivo: str = Field(...)
    conducta: Optional[str] = Field(None)
    
class Resumen(BaseModel):
    id_resume: int = Field(..., description="Identificador del resumen generado por la IA")
    id_paciente: int = Field(..., description="Identificador del paciente")
    resumen: str = Field(..., description="Resumen generado por IA")
    created_at: str = Field(..., description="Fecha de creación del resumen")
    updated_at: str = Field(..., description="Fecha de última actualización del resumen")

class ReferenciaEvolucion(BaseModel):
    id: int = Field(..., description="Identificador de la respuesta generada por la IA")
    id_evolucion: int = Field(..., description="Identificador de la evolución")
    id_resume: int = Field(..., description="Identificador del resumen generado por la IA")

In [9]:
# Seleccion de modelo
llm = init_chat_model("openai:gpt-5-nano")

In [ ]:
# Definir nodo para verificar si el paciente tiene resumen o no
def ejecutar_query_sql(state: int) -> Paciente:
    def query_to_database(query, db):
        # 1. Connect to the SQLite database
        conn = sqlite3.connect(db)

        try:
            df = pd.read_sql_query(query, conn)
            # print(df)
        except sqlite3.Error as e:
            print(f"An error occurred: {e}")
        finally:
            conn.close()
            return df
    
    sql_query_paciente = f"""
    SELECT * FROM paciente WHERE id_paciente = {id_paciente};
    """
    paciente_df = query_to_database(sql_query_paciente, '../01_setup/data/consultorio.db')
    if paciente_df.empty:
        return None
    paciente_dict = paciente_df.iloc[0].to_dict()
    paciente = Paciente(**paciente_dict)
    
    return paciente

In [ ]:
# Definir nodo que 
if paciente is None:
    print(f"No se encontró el paciente con ID {id_paciente}.")
    # Aquí puedes manejar el caso cuando no se encuentra el paciente
else:
    print(f"Paciente encontrado: {paciente.nombre} {paciente.apellido}")
    # Aquí puedes continuar con el procesamiento del paciente

In [ ]:


def obtener_ai_resumen_para_paciente(state: Paciente) -> str:
   def query_to_database(query, db):
    # 1. Connect to the SQLite database
    conn = sqlite3.connect(db)

    try:
        df = pd.read_sql_query(query, conn)
        # print(df)
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
        return df

# Obtener evolucion de un paciente
id_paciente = 172
sql_query_evolucion = f"""
SELECT * FROM evolucion WHERE id_paciente = {id_paciente};
"""
evolucion_df = query_to_database(sql_query_evolucion, '../01_setup/data/consultorio.db')

# Obtener datos personales del paciente
sql_query_paciente = f"""
SELECT * FROM paciente WHERE id_paciente = {id_paciente};
"""

    return { "refined_query": response.content.strip() }

def search_trending_topics(state: TypedDictState) -> TypedDictState:
    refined_query = state['refined_query']

    prompt = f"""
    Given the refined user query, your job is to search for trending topics related to it.

    Refined query: {refined_query}

    Trending topics:"""

    response = llm.bind_tools([tavily_search]).invoke(prompt)

    return { "trending_topics": response.content.strip() }

def analyze_sentiment(state: TypedDictState) -> TypedDictState:

    trending_topics = state['trending_topics']

    prompt = f"""
    Given the trending topics, your job is to analyze their sentiment.

    Trending topics: {trending_topics}

    Sentiment analysis:"""

    response = llm.invoke(prompt)

    return { "sentiment_analysis": response.content.strip() }

builder = StateGraph(TypedDictState)

builder.add_node("refine_user_query", refine_user_query)
builder.add_node("search_trending_topics", search_trending_topics)
builder.add_node("analyze_sentiment", analyze_sentiment)

builder.add_edge(START, "refine_user_query")
builder.add_edge("refine_user_query", "search_trending_topics")
builder.add_edge("search_trending_topics", "analyze_sentiment")
builder.add_edge("analyze_sentiment", END)

graph = builder.compile()

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))